### Import necessary libraries

In [1]:
import pandas as pd
import nltk
import re
import time
import matplotlib.pyplot as plt
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
#nltk.download('wordnet')

### Read data into dataframe

In [2]:
db=pd.read_csv('data/enron_spam_ham_email_clean.csv')
db.head(10)

,Subject,Message,label
0,christmas tree farm pictures,(no_message_text),0
1,"vastar resources , inc .","gary , production from the high island larger ...",0
2,calpine daily gas nomination,- calpine daily gas nomination 1 . doc,0
3,re : issue,fyi - see note below - already done .\nstella\...,0
4,meter 7268 nov allocation,fyi .\n- - - - - - - - - - - - - - - - - - - -...,0
5,mcmullen gas for 11 / 99,"jackie ,\nsince the inlet to 3 river plant is ...",0
6,meter 1517 - jan 1999,"george ,\ni need the following done :\njan 13\...",0
7,duns number changes,fyi\n- - - - - - - - - - - - - - - - - - - - -...,0
8,king ranch,there are two fields of gas that i am having d...,0
9,re : entex transistion,thanks so much for the memo . i would like to ...,0


### Define processing functions

In [3]:
lemmatizer = WordNetLemmatizer()

def regex_processing(msg):
    msg=re.sub('[^a-z\.]'," ",msg.lower()) #lowercasing and removing every character which is not an alphabet or '.'
    msg=re.sub('\s+'," ",msg)              #removing extra whitespaces
    msg=re.sub('\.+\s+\.',".",msg)         #removing double dots to avoid issues during splitting into sentences
    return msg

def sentence_dropping(msg):
    n=2                                    #attempting to drop sentences which don't have more than 2 words in it
    tmp=[sent for sent in sent_tokenize(msg) if len(sent.split())>n]
    if not tmp:
        return msg
    return ' '.join(tmp)

ref_tag={'NN':'n','NNS':'n','NNP':'n','NNPS':'n',
         'VB':'v','VBD':'v','VBG':'v','VBN':'v','VBP':'v','VBZ':'v',
         'JJ':'a','JJR':'a','JJS':'a',
         'RB':'r','RBR':'r','RBS':'r',
         'ADJ_SAT':'s'}

stop_words=set(stopwords.words('english'))

def lemma(msg,def_txt):
    tmp=[]
    for sentence in sent_tokenize(sentence_dropping(regex_processing(msg))):
        tags=nltk.pos_tag(nltk.word_tokenize(sentence))
        lemmatized=' '.join([lemmatizer.lemmatize(word) if ref_tag.get(tag,"")=="" else lemmatizer.lemmatize(word,ref_tag.get(tag)) for word,tag in tags])
        
        filtered=lemmatized
        tmp_var=' '.join([word for word in word_tokenize(lemmatized) if word not in stop_words])
        if len(tmp_var):
            filtered=tmp_var
            
        tmp.append(sentence_dropping(filtered))
    
    if not len(tmp):
        return def_txt
    
    return ' '.join(tmp)

### Process 'Subjects' column

In [4]:
t=time.time()

for i in range(len(db['Subject'])):
    db.loc[i,'Subject']=lemma(db.loc[i,'Subject'],'subject')
    
print(f'Total Elapsed (Subject): {(time.time()-t)//60:.0f} m {(time.time()-t)%60:.0f} s')

Total Elapsed (Subject): 0 m 42 s


### Process 'Messages' column

In [5]:
t=time.time()

for i in range(len(db['Message'])):
    db.loc[i,'Message']=lemma(db.loc[i,'Message'],'message text')
    
print(f'Total Elapsed (Message): {(time.time()-t)//60:.0f} m {(time.time()-t)%60:.0f} s')

db.to_csv("data/enron_spam_ham_email_processed_v2.csv", index=False)

Total Elapsed (Message): 6 m 41 s
